This script is designed to schedule a time-series microcosm experiment in which all microcosms are destructively sampled at the same time, rather than sequentially from t=0. 

This script must be run from a python2.7 conda environment.

REQUIREMENTS
    
    gcalcli    https://github.com/insanum/gcalcli
    
    

In [11]:
import os, re, sys, getopt, datetime

In [13]:
## OBTAIN START DATE INFORMATION FROM USER
def ObtainStart():
    isValid=False
    while not isValid:
        userIn = raw_input("Enter Start Date (dd/mm/yy): ")
        userTime = raw_input("Enter Start Time (HH:MM) : ")
        
        try: # strptime throws an exception if the input doesn't match the pattern
            d = datetime.datetime.strptime(userIn, "%d/%m/%y")
            t = datetime.datetime.strptime(userTime, "%H:%M")
            c = datetime.datetime.combine(d.date(), t.time())
            
            isValid=True
        except:
            print "Doh, try again!\n"
    return c

Type Date dd/mm/yy: 28/02/17
Time HH:MM : 21:30
2017-02-28 21:30:00


In [18]:
## OBTAIN TIMEPOINT INFORMATION
def ObtainTimePoints():
    isValid=False
    
    while not isValid:
        points = raw_input("Enter Time Points (4,7,14,32,48): ")
        unit = raw_input("Enter Unit of Time (minutes | hours | days | months): ")
        
        try: # strptime throws an exception if the input doesn't match the pattern
            points = points.strip()
            timepoints = points.split(",")
            
            print "You have scheduled "+str(len(timepoints))+" timepoints, on the time-scale of "+unit
            isValid=True
        except:
            print "Doh, try again!\n"
    return timepoints, unit

Enter Time Points (4,7,14,32,48): 12,24,36
Enter Unit of Time (minutes | hours | days | months): hours
You have scheduled 3 timepoints, on a time-scale of hours
(['12', '24', '36'], 'hours')


In [7]:

#str(datetime.date.today() + relativedelta(days=+5))

#os.system('gcalcli agenda')


NameError: name 'ObtainDate' is not defined

In [ ]:

START = ObtainStart()
Timing = ObtainTimePoints()